# Exercise 1: Elastic rotating bar

In [1]:
import ngsolve
if ngsolve.__version__ < "6.2.2405-28": 
    raise Exception("NGSolve too old, pls run 'pip3 install --upgrade --pre ngsolve'")

In [2]:
from ngsolve import *
from netgen.occ import *
import ipywidgets as widgets
from ngsolve.comp import GlobalSpace
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton
from ngsolve.comp import DifferentialSymbol
import numpy as np
# ngsglobals.msg_level=10
import solvers

In [3]:
shape = Rectangle(1, 0.1).Face()

# move shape's center of mass to origin; set start translation accordingly
r = (shape.center[0], shape.center[1])
shape = shape.Move((-r[0], -r[1], 0))

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [4]:
moment_of_inertia = Integrate((x**2 + y**2)*dx, mesh)
mass = Integrate(CF(1)*dx, mesh)

## time-stepping

Variables are

* position $\phi = r + R x$, an affine linear function, with constraint $R^T R = I$, i.e. $R$ is orthogonal (and thus, by continuity, a rotation matrix)
* velocity $v = a + b \wedge x$, in body-frame

In [5]:
force = lambda q, v: np.array( (0,0.0001) )

# r...translation

mass_matrix = np.array([[mass, 0],
                        [0, mass]
                       ])

fes = VectorH1(mesh, order=2)
gf_deformation = GridFunction(fes)
gf_deformation.Set(CF((0, 0)))

def callback(current_step, new, error):
    # new is current state: r_1, r_2, v_1, v_2
    r = new[0:2]
    
    gf_deformation.Set(CF(tuple(r)))
    scene.Redraw()
    print(r)
    # tw.value = F"t = {t:.2f}"
    

In [6]:
scene = Draw (gf_deformation, mesh, deformation=(gf_deformation-CF((x,y))), center=(0,0,0), radius=1.2)

solvers.newmark(q0=np.array(r), v0=np.array((0, 0)), M=mass_matrix, C=np.zeros((2, 2)), force=force, steps=100, t_end=6, callback=callback)

# tw = widgets.Text(value='t = 0')
# display(tw)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

[0.5       0.0500009]
[0.5       0.0500045]
[0.5       0.0500117]
[0.5       0.0500225]
[0.5       0.0500369]
[0.5       0.0500549]
[0.5       0.0500765]
[0.5       0.0501017]
[0.5       0.0501305]
[0.5       0.0501629]
[0.5       0.0501989]
[0.5       0.0502385]
[0.5       0.0502817]
[0.5       0.0503285]
[0.5       0.0503789]
[0.5       0.0504329]
[0.5       0.0504905]
[0.5       0.0505517]
[0.5       0.0506165]
[0.5       0.0506849]
[0.5       0.0507569]
[0.5       0.0508325]
[0.5       0.0509117]
[0.5       0.0509945]
[0.5       0.0510809]
[0.5       0.0511709]
[0.5       0.0512645]
[0.5       0.0513617]
[0.5       0.0514625]
[0.5       0.0515669]
[0.5       0.0516749]
[0.5       0.0517865]
[0.5       0.0519017]
[0.5       0.0520205]
[0.5       0.0521429]
[0.5       0.0522689]
[0.5       0.0523985]
[0.5       0.0525317]
[0.5       0.0526685]
[0.5       0.0528089]
[0.5       0.0529529]
[0.5       0.0531005]
[0.5       0.0532517]
[0.5       0.0534065]
[0.5       0.0535649]
[0.5      

(array([0.5      , 0.0678209, 0.       , 0.00597  , 0.       , 0.001    ]),
 100)

In [7]:
help(CF)

Help on class CoefficientFunction in module ngsolve.fem:

class CoefficientFunction(pybind11_builtins.pybind11_object)
 |  A CoefficientFunction (CF) is some function defined on a mesh.
 |  Examples are coordinates x, y, z, domain-wise constants, solution-fields, ...
 |  CFs can be combined by mathematical operations (+,-,sin(), ...) to form new CFs
 |  Parameters:
 |
 |  val : can be one of the following:
 |
 |    scalar (float or complex):
 |      Creates a constant CoefficientFunction with value val
 |
 |    tuple of scalars or CoefficientFunctions:
 |      Creates a vector or matrix valued CoefficientFunction, use dims=(h,w)
 |      for matrix valued CF
 |    list of scalars or CoefficientFunctions:
 |      Creates a domain-wise CF, use with generator expressions and mesh.GetMaterials()
 |      and mesh.GetBoundaries()
 |
 |  Method resolution order:
 |      CoefficientFunction
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  Com